In [1]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.2 MB/s 
     |████████████████████████████████| 6.6 MB 58.0 MB/s 
     |████████████████████████████████| 120 kB 61.1 MB/s 
     |████████████████████████████████| 1.3 MB 58.0 MB/s 


In [2]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
checkpoint = "papluca/xlm-roberta-base-language-detection"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [4]:
df_all_artists_raw = pd.read_csv("df_all_artists_raw.csv")

df_all_artists_raw.tail()

,Unnamed: 0,Unnamed: 0.1,Artist,Title,Album,Year,Date,Lyric
6022,59,59.0,Khalid,Khalid - Vertigo (Tradução Português),NaN,2018.0,2018-10-28,será que é melhor apenas acreditar nas teorias...
6023,60,60.0,Khalid,Better (Miles Away Remix),NaN,2018.0,2018-12-12,i'm not really drunk i never get that fucked u...
6024,61,61.0,Khalid,Khalid - Better (Official Music Video),NaN,2018.0,2018-05-07,users considering it's a virus or malware must...
6025,62,62.0,Khalid,Perfect Lover,NaN,NaN,NaN,lyrics for this song have yet to be released p...
6026,63,63.0,Khalid,Better (Rennie! Remix),NaN,2019.0,2019-02-01,love to see you shine in the night like the di...


In [5]:
df_all_artists_lyrics_present = df_all_artists_raw[df_all_artists_raw.Lyric.notnull()]

In [6]:
sequences = list(df_all_artists_lyrics_present['Lyric'].str[:200])
sequences[:5]

["one one one one one   talkin' in my sleep at night makin' myself crazy out of my mind out of my mind wrote it down and read it out hopin' it would save me too many times too many times  refrain my lov",
 "if you don't wanna see me   did a full 80 crazy thinking 'bout the way i was did the heartbreak change me maybe but look at where i ended up i'm all good already so moved on it's scary i'm not where y",
 "you call me all friendly tellin' me how much you miss me that's funny i guess you've heard my songs well i'm too busy for your business go find a girl who wants to listen 'cause if you think i was bor",
 "i know it's hot i know we've got something that money can't buy fighting in fits biting your lip loving 'til late in the night  pre tell me i'm too crazy you can't tame me can't tame me tell me i have",
 "i see the moon i see the moon i see the moon oh when you're looking at the sun i'm not a fool i'm not a fool not a fool no you're not fooling anyone  pre oh but when you're gone 

In [7]:
def generate_batch(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i : i + batch_size]

In [8]:
# batch = [
#     "It's such a wonderful world.",
#     "Estoy muy contenta.",
# ]

batch_size = 50
input_batches = generate_batch(sequences, batch_size)
lyric_language = []

for batch in input_batches:
  model_inputs = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
  output = model(**model_inputs)
  predictions = torch.nn.functional.softmax(output.logits, dim=-1)
  ranking = np.argmax(predictions.detach().numpy(), axis=1)
  lyric_language.append([model.config.id2label[i] for i in ranking])

In [9]:
lyric_language = [item for sublist in lyric_language for item in sublist]
lyric_language[:5]

['en', 'en', 'en', 'en', 'en']

In [10]:
df_all_artists_lyrics_present['lang'] = lyric_language
df_all_artists_lyrics_present.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,Unnamed: 0.1,Artist,Title,Album,Year,Date,Lyric,lang
6022,59,59.0,Khalid,Khalid - Vertigo (Tradução Português),NaN,2018.0,2018-10-28,será que é melhor apenas acreditar nas teorias...,pt
6023,60,60.0,Khalid,Better (Miles Away Remix),NaN,2018.0,2018-12-12,i'm not really drunk i never get that fucked u...,en
6024,61,61.0,Khalid,Khalid - Better (Official Music Video),NaN,2018.0,2018-05-07,users considering it's a virus or malware must...,en
6025,62,62.0,Khalid,Perfect Lover,NaN,NaN,NaN,lyrics for this song have yet to be released p...,en
6026,63,63.0,Khalid,Better (Rennie! Remix),NaN,2019.0,2019-02-01,love to see you shine in the night like the di...,en


In [11]:
import collections
collections.Counter(lyric_language)

Counter({'en': 5018,
         'sw': 669,
         'ja': 186,
         'fr': 5,
         'es': 24,
         'hi': 14,
         'nl': 5,
         'th': 1,
         'ur': 26,
         'ru': 3,
         'tr': 10,
         'zh': 1,
         'pl': 5,
         'it': 7,
         'de': 1,
         'el': 2,
         'pt': 4})

In [12]:
df_all_artists_lyrics_present[df_all_artists_lyrics_present['lang']=='pl']

,Unnamed: 0,Unnamed: 0.1,Artist,Title,Album,Year,Date,Lyric,lang
1578,59,59.0,Cardi B,Earthquake*,NaN,NaN,NaN,shake earthquake wheezy outta here,pl
2090,496,496.0,Eminem,Premonition (PL),NaN,NaN,NaN,eminem tak więc chyba tak to jest co myślę że ...,pl
4564,477,477.0,Taylor Swift,Tolerate it (Polskie Tłumaczenie),NaN,2020.0,2020-12-11,zwrotka siedzę i patrzę jak czytasz z głową p...,pl
5426,341,341.0,Rihanna,Disturbia (Jody den Broeder Bum Bum Dub),Disturbia (Remixes),2008.0,2008-08-05,w background vocals,pl
5455,370,370.0,Rihanna,Disturbia (Craig C’s Disturbstramental Mix),Disturbia (Remixes),2008.0,2008-08-05,w background vocals,pl


In [14]:
df_all_artists_lyrics_present.to_csv('df_all_lyrics_with_lang.csv', sep='|')